In [1]:
#importer les librairies
import pandas as pd
import re
import sqlparse
import sqlparser
import pyparsing
import glob


# Création du dataframe

In [2]:
#créer le dataframe à partir d'un fichier csv
df=pd.read_csv("sql_requetes.csv", sep=";")

In [3]:
#lire toutes les infos du df
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None)

In [4]:
#lire les 10 premières lignes du dataframe
df.head(10)

,projet,requete_sql
0,ATP,"select * from (select nom, adresse from Immeub..."
1,ATP,SELECT * FROM Table
2,ATP,SELECT * FROM question WHERE q_id IN (SELECT r...
3,ATP,SELECT sum(a.Population)FROM travail.communes6...
4,ATP,"SELECT timestamp, log_id, severity, json_paylo..."
5,ATP,SELECT * FROM Orders WHERE CustomerID IN (SEL...
6,ATP,SELECT COUNT(count_table.mois) FROM(SELECT st....
7,PLE,"SELECT \tTDI_D01 as TDI_D01, \tTDI_D02 as TDI..."
8,PLE,"SELECT \tTDI_D01 as TDI_D01, \tTDI_D02 as TDI..."
9,PLE,"Select * FROM T1,T2 Where T1.ID=T2.FK_ID"


# Création regex pour extraire nom de table dans requete SQL

In [5]:
def extraire_noms_tables(requete_sql):
    # Utilisation d'une expression régulière pour trouver les occurrences de motifs de nom de table
    motif = r'\bFROM\s+([a-zA-Z_][a-zA-Z0-9_]*)\b|\bJOIN\s+([a-zA-Z_][a-zA-Z0-9_]*)\b|\bINTO\s+([a-zA-Z_][a-zA-Z0-9_]*)\b'
    matches = re.findall(motif, requete_sql, flags=re.IGNORECASE)

    # Regrouper les résultats de toutes les expressions régulières
    noms_tables = [table for match in matches for table in match if table]

    return noms_tables
"""
# Exemple d'utilisation
requete_sql = "SELECT * FROM users JOIN orders ON users.id = orders.user_id WHERE users.active = 1"
tables = extraire_noms_tables(requete_sql)

print("Noms des tables :", tables)
"""

'\n# Exemple d\'utilisation\nrequete_sql = "SELECT * FROM users JOIN orders ON users.id = orders.user_id WHERE users.active = 1"\ntables = extraire_noms_tables(requete_sql)\n\nprint("Noms des tables :", tables)\n'

## Création regex pour extraire tables dans version sql plus ancienne

In [6]:
def extract_old_sql(requete_sql):
    # Recherche de l'indice du mot "FROM"
    index_from = requete_sql.upper().find("FROM")
    
    if index_from != -1:
        # Extrait la sous-chaîne après le mot "FROM"
        sous_chaine = requete_sql[index_from + 4:]
        
        # Supprime les espaces en début et en fin de chaîne
        sous_chaine = sous_chaine.strip()
        
        # Recherche de l'indice du mot "WHERE"
        index_where = sous_chaine.upper().find("WHERE")
        
        if index_where != -1:
            # Extrait la sous-chaîne avant le mot "WHERE"
            sous_chaine_tables = sous_chaine[:index_where].strip()
            
            # Vérifie s'il y a une virgule dans la sous-chaîne
            if ',' in sous_chaine_tables:
                # Sépare les noms de table en fonction des virgules
                noms_tables = [table.strip() for table in sous_chaine_tables.split(',')]
                return noms_tables

# Exemples d'utilisation avec vos requêtes
requete1 = "Select * FROM T1,T2 Where T1.ID=T2.FK_ID"
noms_tables1 = extraire_noms_tables(requete1)
print(noms_tables1)  # None, car pas de virgule après FROM

requete2 = "Select * FROM T1,T2,T3 Where T1.ID=T2.FK_ID"
noms_tables2 = extraire_noms_tables(requete2)
print(noms_tables2)  # ['T1', 'T2', 'T3'], car il y a une virgule après FROM

requete3 = "Select * FROM T1 Where T1.ID=T2.FK_ID"
noms_tables3 = extraire_noms_tables(requete3)
print(noms_tables3)  # ['T1', 'T2', 'T3', 'T4'], car il y a une virgule après FROM


['T1']
['T1']
['T1']


## Création du dataframe avec les tables

In [7]:
# Appliquer la fonction personnalisée à chaque ligne de la colonne 'requete_sql'
df['table'] = df['requete_sql'].apply(extraire_noms_tables)

# S'assurer que la colonne 'table' est de type chaîne
df['table'] = df['table'].astype(str)

# Supprimer les crochets de la colonne 'table'
df['table']=df['table'].str.replace('[', '').str.replace(']', '')

# Supprimer tous les guillemets simples de la colonne 'table'
df['table'] = df['table'].str.replace('\'', '')


In [8]:
# Appliquer la fonction personnalisée à chaque ligne de la colonne 'requete_sql'
df['table_2'] = df['requete_sql'].apply(extract_old_sql)

# S'assurer que la colonne 'table' est de type chaîne
df['table_2'] = df['table_2'].astype(str)

# Supprimer les crochets de la colonne 'table'
df['table_2']=df['table_2'].str.replace('[', '').str.replace(']', '')

# Supprimer tous les guillemets simples de la colonne 'table'
df['table_2'] = df['table_2'].str.replace('\'', '')


In [9]:
def mettre_none_si_commence_par_parenthese(valeur):
    if isinstance(valeur, str) and valeur.startswith("("):
        return None
    else:
        return valeur

# Appliquer la fonction à la colonne 'table_2'
df['table_2'] = df['table_2'].apply(mettre_none_si_commence_par_parenthese)

# Remplacer les valeurs nulles (None) dans la colonne "table_2" par celles de la colonne "table"
df['table_2'] = df['table_2'].apply(lambda x: x if x != 'None' else None)
df['table_2'].fillna(df['table'], inplace=True)

# Supprimer la colonne "table"
df.drop('table', axis=1, inplace=True)

# Afficher le DataFrame résultant
df.tail(10)

,projet,requete_sql,table_2
0,ATP,"select * from (select nom, adresse from Immeub...",Immeuble
1,ATP,SELECT * FROM Table,Table
2,ATP,SELECT * FROM question WHERE q_id IN (SELECT r...,"question, reponse"
3,ATP,SELECT sum(a.Population)FROM travail.communes6...,"travail, travail"
4,ATP,"SELECT timestamp, log_id, severity, json_paylo...",TABLE
5,ATP,SELECT * FROM Orders WHERE CustomerID IN (SEL...,"Orders, Customers, Shippers, OrderDetails, Pro..."
6,ATP,SELECT COUNT(count_table.mois) FROM(SELECT st....,"t, transaction, city, somme_transactions_mois"
7,PLE,"SELECT \tTDI_D01 as TDI_D01, \tTDI_D02 as TDI...","FUS_CONTENTIEUX, SIM_GCC_CLIENTS, SIM_GCD_CONT..."
8,PLE,"SELECT \tTDI_D01 as TDI_D01, \tTDI_D02 as TDI...","FUS_CONTENTIEUX, SIM_GCC_CLIENTS, SIM_GCD_CONT..."
9,PLE,"Select * FROM T1,T2 Where T1.ID=T2.FK_ID","T1, T2"


## Test sur un fichier Excel extrait de Talend

In [17]:
#créer le dataframe à partir d'un fichier csv
df=pd.read_excel("C:/Talend/TOS_DI-8.0.1/studio/workspace/ATP_requete.xlsx")
df.head()

SyntaxError: invalid syntax (394037191.py, line 2)

In [11]:
# Appliquer la fonction personnalisée à chaque ligne de la colonne 'requete_sql'
df['table'] = df['requete_sql'].apply(extraire_noms_tables)

# S'assurer que la colonne 'table' est de type chaîne
df['table'] = df['table'].astype(str)

# Supprimer les crochets de la colonne 'table'
df['table']=df['table'].str.replace('[', '').str.replace(']', '')

# Supprimer tous les guillemets simples de la colonne 'table'
df['table'] = df['table'].str.replace('\'', '')

# Appliquer la fonction personnalisée à chaque ligne de la colonne 'requete_sql'
df['table_2'] = df['requete_sql'].apply(extract_old_sql)

# S'assurer que la colonne 'table' est de type chaîne
df['table_2'] = df['table_2'].astype(str)

# Supprimer les crochets de la colonne 'table'
df['table_2']=df['table_2'].str.replace('[', '').str.replace(']', '')

# Supprimer tous les guillemets simples de la colonne 'table'
df['table_2'] = df['table_2'].str.replace('\'', '')

df.head(20)


,id_job,componentName,requete_sql,table,table_2
0,NaN,tPostgresqlRow,truncate table job_referentiel,,None
1,NaN,tPostgresqlRow,truncate table job_hierarchy,,None
2,NaN,tPostgresqlRow,truncate table job_table,,None
3,NaN,tPostgresqlRow,truncate table job_sql_table,,None
4,NaN,tPostgresqlRow,truncate table job_load_context,,None
5,NaN,tPostgresqlRow,truncate table job_variable,,None
6,NaN,tPostgresqlRow,truncate table job_group_context,,None
7,NaN,tPostgresqlRow,truncate table err_process_sans_properties,,None
8,NaN,tPostgresqlRow,truncate table job_hierarchy_level,,None
9,NaN,tPostgresqlRow,truncate table job_plan_tac,,None


In [12]:
def mettre_none_si_commence_par_parenthese(valeur):
    if isinstance(valeur, str) and valeur.startswith("("):
        return None
    else:
        return valeur

# Appliquer la fonction à la colonne 'table_2'
df['table_2'] = df['table_2'].apply(mettre_none_si_commence_par_parenthese)

# Remplacer les valeurs nulles (None) dans la colonne "table_2" par celles de la colonne "table"
df['table_2'] = df['table_2'].apply(lambda x: x if x != 'None' else None)
df['table_2'].fillna(df['table'], inplace=True)

# Supprimer la colonne "table"
df.drop('table', axis=1, inplace=True)

# Afficher le DataFrame résultant
df.head(50)

,id_job,componentName,requete_sql,table_2
0,NaN,tPostgresqlRow,truncate table job_referentiel,
1,NaN,tPostgresqlRow,truncate table job_hierarchy,
2,NaN,tPostgresqlRow,truncate table job_table,
3,NaN,tPostgresqlRow,truncate table job_sql_table,
4,NaN,tPostgresqlRow,truncate table job_load_context,
5,NaN,tPostgresqlRow,truncate table job_variable,
6,NaN,tPostgresqlRow,truncate table job_group_context,
7,NaN,tPostgresqlRow,truncate table err_process_sans_properties,
8,NaN,tPostgresqlRow,truncate table job_hierarchy_level,
9,NaN,tPostgresqlRow,truncate table job_plan_tac,


In [13]:
df_test=df['table_2']

In [14]:
df_test

0                                   
1                                   
2                                   
3                                   
4                                   
5                                   
6                                   
7                                   
8                                   
9                                   
10                                  
11    job_referentiel, job_hierarchy
12    job_referentiel, job_hierarchy
13        job_hierarchy_level_x000D_
14                     job_hierarchy
15                      job_plan_tac
16                   talend_tracking
17                   talend_tracking
18            job_referentiel_x000D_
19                   talend_tracking
20                   talend_tracking
21                   talend_tracking
22                   talend_tracking
23                                  
Name: table_2, dtype: object

In [15]:
df.to_csv("C:/Talend/TOS_DI-8.0.1/studio/workspace/job_tables_sql.csv")
df_test.to_csv("C:/Talend/TOS_DI-8.0.1/studio/workspace/test_sql.csv")

## Combiner plusieurs fichiers

In [23]:
# Chemin du répertoire contenant les fichiers
chemin_repertoire = "C:/Talend/TOS_DI-8.0.1/studio/workspace/"

# Modèle pour les fichiers que vous souhaitez lire
modele_fichier = "*_requete.xlsx"

# Utiliser glob pour obtenir la liste des fichiers correspondants au modèle
chemin_fichiers = glob.glob(chemin_repertoire + modele_fichier)

# Initialiser un DataFrame vide pour stocker les données de tous les fichiers
df_global = pd.DataFrame()

# Boucle pour lire chaque fichier, ajouter une colonne avec le nom du fichier, et concaténer les données
for fichier in chemin_fichiers:
    df_temp = pd.read_excel(fichier)
    
    # Extraire le nom du fichier (sans le chemin)
    nom_fichier = fichier.split("\\")[-1]
    nom_fichier= nom_fichier.split(".")[0]
    
    # Ajouter une colonne "Nom_Fichier" au DataFrame temporaire
    df_temp = df_temp.assign(projet=nom_fichier)
    
    # Concaténer les données dans le DataFrame global
    df_global = pd.concat([df_global, df_temp], ignore_index=True)

# Afficher les premières lignes du DataFrame global
df_global.head(50)


,id_job,componentName,requete_sql,projet
0,NaN,tPostgresqlRow,truncate table job_referentiel,ATP_requete
1,NaN,tPostgresqlRow,truncate table job_hierarchy,ATP_requete
2,NaN,tPostgresqlRow,truncate table job_table,ATP_requete
3,NaN,tPostgresqlRow,truncate table job_sql_table,ATP_requete
4,NaN,tPostgresqlRow,truncate table job_load_context,ATP_requete
5,NaN,tPostgresqlRow,truncate table job_variable,ATP_requete
6,NaN,tPostgresqlRow,truncate table job_group_context,ATP_requete
7,NaN,tPostgresqlRow,truncate table err_process_sans_properties,ATP_requete
8,NaN,tPostgresqlRow,truncate table job_hierarchy_level,ATP_requete
9,NaN,tPostgresqlRow,truncate table job_plan_tac,ATP_requete
